In [ ]:
from tqdm import tqdm
import torch
from transformers import AutoTokenizer
import models
# 这里写你要测试的模型名称 （之前预训练好的）
model_save_name = ''
test_data = ''

all_keys = 'key, data_type, showtxt, pcurl, title, direct_answer, content, wapurl, pic, from_url, icon, voice, time, piclist, logopic, score, key_info'
all_QA_pair = []
all_contents = []
with open(test_data,'r',encoding='utf-8') as f:
    all_file = f.readlines()
    keys = all_keys.split(', ')
    for line in all_file:
        line = line.strip()
        all_item = line.split('\t')
        current_dic = dict(zip(keys, all_item))
        all_contents.append(current_dic)
        question = current_dic['key']
        content = current_dic['content']
        all_QA_pair.append({'question':question, 'content':content})


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ROBERTA_QA_model = models.ROBERTA_QA_model(model_name='hfl/chinese-roberta-wwm-ext-large').to(device)
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext-large')
ROBERTA_QA_model = torch.load('Model/'+  model_save_name)
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
cls_token = tokenizer.cls_token
all_token = [sep_token,pad_token,cls_token]
def test_with_roberta(data, model):
    pred_word =  []
    model.eval()
    with torch.no_grad():
        for pair in tqdm(data):
            question = pair['question']
            paragraph = pair['content']
            tokenized_sentence = self.tokenizer.tokenize(paragraph, question)
            original_sentence = paragraph + question
            temp = 0
            actual_sentence = []
            for index, word in enumerate(tokenized_sentence):
                if word != '[UNK]':
                    actual_sentence.append(word)
                else:
                    word = original_sentence[temp:temp + 2]
                    actual_sentence.append(word)
                temp += len(word)
            token_dic =tokenizer(paragraph, question, max_length=self.MAX_SEQ_LEN, padding='max_length', truncation=True)
            input_ids_final = token_dic['input_ids'].to(device)
            attention_mask = token_dic['attention_mask'].to(device)
            segment_ids = token_dic['token_type_ids'].to(device)
            start_logits, end_logits = model(input_ids=input_ids_final,
                                     attention_mask=attention_mask, token_type_ids=segment_ids)

            start = torch.argmax(start_logits, dim=1).tolist()
            end = torch.argmax(end_logits, dim=1).tolist()
            if int(end) == int(start) and int(end) == 0:
                pred_word.append('')
            else:
                pre_ans = ''.join(actual_sentence[int(start)-1:int(end)])
                pred_word.append(replace(pre_ans, all_token))

    return pred_word

all_prediction = test_with_roberta(all_QA_pair, ROBERTA_QA_model)
with open('test_data1210_prediction.txt', 'w', encoding='utf-8') as f:
    for index, line in tqdm(enumerate(all_contents)):
        line['answer'] = all_prediction[index]
        final_text = '\t'.joint(line.values())
        f.write(final_text)
        f.write('\n')

In [3]:
import json
from tqdm import tqdm
import pandas as pd

def find_lcsubstr(str_a, str_b):
#  longest common subsequence of str_a and str_b, with O(n) space complexity
    if len(str_a) == 0 or len(str_b) == 0:
        return 0
    dp = [0 for _ in range(len(str_b) + 1)]
    for i in range(1, len(str_a) + 1):
        left_up = 0
        dp[0] = 0
        for j in range(1, len(str_b) + 1):
            left = dp[j-1]
            up = dp[j]
            if str_a[i-1] == str_b[j-1]:
                dp[j] = left_up + 1
            else:
                dp[j] = max([left, up])
            left_up = up
    return dp[len(str_b)]/min(len(str_b),len(str_a))

def data_loading(path):
    all_keys = ['question_id','question','qas_id','doc_tokens','answer_text','start_position','end_position','is_impossible']
    all_values = [[] for i in range(len(all_keys))]
    final_dic = dict(zip(all_keys,all_values))
    with open(path,encoding='utf-8') as json_file:
        for line in tqdm(json_file):
            data = json.loads(line)
            current_keys = data.keys()
            for key in current_keys:
                final_dic[key].append(data[key])
    final_pd = pd.DataFrame(final_dic)
    final_pd.start_position = final_pd.start_position.replace({-1:0})
    final_pd.end_position = final_pd.end_position.replace({-1:0})
    return final_pd

test_df_location = 'cleaned_data_test.json'
evaluate_file = '1130_roberta_3.0data.json'
test_df = data_loading(test_df_location)
all_qas_id = list(test_df['qas_id'])
all_question = list(test_df['question'])
all_doc_tokens = list(test_df['doc_tokens'])
answer_all = list(test_df['answer_text'])
final_qas_map = dict(zip(all_qas_id,answer_all))
final_que_map = dict(zip(all_qas_id,all_question))
final_doc_map = dict(zip(all_qas_id,all_doc_tokens))
shot_ans = ['是什么','在哪里','哪位','哪国','国家','国籍','哪个朝代','本名','原名','哪个','哪一个','哪','最','几次','是谁','学名','第一个','第一','最大','最小','最长','最深','最高','最多','第一','作者','叫什么名字','名字','叫什么']

1500it [00:00, 93606.14it/s]


In [12]:
import jieba

def check_relevance(question, answer, do_filter=False, threshould = 0.2):  # do_filter表示是否返回合格答案 True则返回 False只返回判断
    check_list = ['建议','当地','需要','确定','咨询','询问','客服','相关','部门','有关部门','私信','同求','您好','办理','手续','及时']
    right_list = ['这种情况','情况','病史','描述','叙述','根治','缓解','改善','症状','表现','食用','口服','服用','导致','表现','形容',
    '危险','根据你的描述','引起的','根据','正常','可能是','你说的','导致的','造成的','检查一下','试试','尝试','进行治疗','检查','治疗',
    '药物治疗','治疗中','相关治疗','治疗方式','治疗方案','治疗效果','常规治疗','其他治疗','治疗后','手术治疗','治疗方法','综合治疗',
    '系统治疗','治疗手段','治疗方面','中医治疗','治疗措施','治疗过程中','疾病的治疗','对症治疗','持续治疗','临床治疗','其它治疗',
    '西医治疗','保守治疗','医治','长期治疗','治疗效果不佳','一般治疗','治疗法']
    flag = 0 # flag = 1 代表这个数据中存在合格答案 （长度大于20且无关键词）
    mark = 0
    valid_answer = ''
    sentence = answer
    current_sentence = [i for i in jieba.cut(sentence, cut_all=False)]
    current_question = [i for i in jieba.cut(question, cut_all=False)]
#         current_sentence = [i.term for i in wordseg.segment(sentence).basic_words]
#         current_question = [i.term for i in wordseg.segment(question).basic_words]
    for item in right_list:
        if item in sentence:
            flag = 1
            valid_answer = sentence if do_filter else 0
            break
    if flag == 1 and do_filter:
        return valid_answer, flag
    elif flag == 1 and not do_filter:
        return flag
    
    else:
        if len(current_sentence) <= 20: # 长度小于20的情况
            for item in check_list:
                if item in sentence:
                    mark = 1
                    break
            if mark == 1 and do_filter: # 包含关键词不考虑
                return valid_answer, flag
            elif mark == 1 and not do_filter:
                return flag
            else: # 小于20单不包含关键词 则判断问题答案重叠率
                final_q = set(current_question)
                candidate_all = final_q.intersection(set(current_sentence))
                if len(candidate_all)/len(final_q) >= threshould:
                    flag = 1
                    valid_answer = sentence

        elif len(current_sentence) > 20:  # 长度大于20的情况
            final_q = set(current_question)
            candidate_all = final_q.intersection(set(current_sentence))
            if len(candidate_all)/len(final_q) >=threshould: #重叠率判断
                flag = 1
                valid_answer = sentence
                
    if do_filter:
        return valid_answer, flag
    else:
        return flag

In [15]:
def load_prediction_RoBERTA_short(path, final_qas_map,final_que_map, final_doc_map, short_ans):
    final_text=[]
    final_answer = []
    final_qid = []
    final_text2=[]
    ground_truth = []
    final_answer2 = []
    final_qid2 = []
    prediction_result_100 = []
    prediction_result_60 = []
    prediction_not_null_60 = []
    prediction_not_null_100 = []
    count = 0
    with open(path, "r",encoding='utf-8') as f:
        all_data = f.readlines()
        for line in tqdm(all_data):
            mark = 0
            current_object = json.loads(line)
            qas_id = current_object['question_ID']
            question = final_que_map[qas_id]
            for q in shot_ans:
                if q in question:
                    mark = 1
                    break
#             actual_answer = ''.join(current_object['answer'].split())
            if mark == 1:
                actual_answer = ''.join(current_object['answer'].split())
                actual_answer_final = final_qas_map[qas_id]
                prediction = ''.join(current_object['prediction'].split())
                
                relevancy = check_relevance(question, prediction)
                if actual_answer == prediction and actual_answer == '':
                    prediction_result_100.append(1)
                    prediction_result_60.append(1)
                else:
                    overlap_rate = find_lcsubstr(actual_answer, prediction)
                    if overlap_rate == 1: # 考虑包含情况
                        prediction_result_100.append(1)
                    else:
                        prediction_result_100.append(0)
                    if overlap_rate >=0.6: #考虑包含情况
                        prediction_result_60.append(1)
                    else:
                        prediction_result_60.append(0)
    #             (prediction in answer and len(prediction)/len(answer)>=0.5)
                if actual_answer_final!='':
                    count +=1
                if actual_answer!='':
                    overlap_rate = find_lcsubstr(actual_answer, prediction)
                    if overlap_rate == 1:
                        prediction_not_null_100.append(1)
                    else:
                        prediction_not_null_100.append(0)
                    if overlap_rate >= 0.6:
                        prediction_not_null_60.append(1)
                    else:
                        prediction_not_null_60.append(0)
                
                if actual_answer == '' and prediction != '':
                    text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                    final_text.append(text)
                    final_answer.append(prediction)
                    final_qid.append(qas_id)

                if actual_answer != '' and prediction != actual_answer:
                    text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                    final_text2.append(text)
                    ground_truth.append(actual_answer_final)
                    final_answer2.append(prediction)
                    final_qid2.append(qas_id)
        print('RoBERTa shot_answer prediction, total length:{}'.format(len(prediction_result_100)))
        print('100% 相同情况下 准确率：', prediction_result_100.count(1)/len(prediction_result_100))
        print('60% 相同情况下 准确率：', prediction_result_60.count(1)/len(prediction_result_60))
        print('非空总数：{}'.format(count))
        print('100% 相同情况下 答案非空 准确率：', prediction_not_null_100.count(1)/len(prediction_not_null_100))
        print('60% 相同情况下 答案非空 准确率：', prediction_not_null_60.count(1)/len(prediction_not_null_60))
        final_dic = list(zip(final_qid, final_text, final_answer))
        final_dic2 = list(zip(final_qid2, final_text2, final_answer2, ground_truth))
        return final_dic,final_dic2

In [16]:
suspicious_ROBERTA_short, nonconsistent_ROBERTA_short = load_prediction_RoBERTA_short(evaluate_file, final_qas_map,final_que_map, final_doc_map,shot_ans)

100%|████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 3962.68it/s]

RoBERTa shot_answer prediction, total length:575
100% 相同情况下 准确率： 0.9582608695652174
60% 相同情况下 准确率： 0.9652173913043478
非空总数：497
100% 相同情况下 答案非空 准确率： 0.9655172413793104
60% 相同情况下 答案非空 准确率： 0.973630831643002


In [17]:
suspicious_ROBERTA_short

[('N520408',
  'Paragraph:红魔电竞游戏手机是首款针对游戏玩家打造的机型，该产品于2018年4月19日发布。该款电竞游戏手机搭载了高通骁龙835处理器，拥有6GB+64GB和8GB+128GB两个储存版本，配备了6英寸分辨率为2160乘以1080的全面屏。.Question:红魔是什么牌子的手机',
  '红魔'),
 ('N1189940',
  'Paragraph:1、《红岩》的作者是国民党集中营的幸存者。1961年12月，正式出版，此书重印113次，再版2次，册数则超过了一千万。作为红色经典作品，轰动一时。2、该书讲述了1948年在国民党的统治下，处在黎明前最黑暗的时刻的共产党员在渣滓洞和白公馆中受刑，敌人为了得到口供，妄图用炎热、蚊虫、饥饿和干渴动摇革命者的意志，但在共产党员的坚强意志前，敌人却是一筹莫展，一败涂地的故事。.Question:红岩的作者是谁',
  '《红岩》的作者是国民党集中营的幸存者'),
 ('N577874',
  'Paragraph:鱼舌头，它的舌头在口腔底部，除了可以感受味觉，还能帮助吞咽食物。它的舌头没弹性，而且肌肉不发达，基本上不能活动。这类动物的体形四种，其中纺锤形最为普遍，非常善于游泳，侧扁型比较窄，平扁型比较薄，而棍棒型像蛇一样，这三种的游泳能力都不强。.Question:哪些鱼有舌头',
  '鱼舌头'),
 ('N534111',
  'Paragraph:知周长求直径的公式是直径是指通过一平面图形或立体中心到边上两点间的距离，通常用字母“d”表示。连接圆周上两点并通过圆心的线段称圆直径，连接球面上两点并通过球心的直线称球直径。直径的性质有：在同一个圆中直径的长度是半径的2倍，可以表示d=2r或r=d/2。在同一个圆中直径是最长的弦。直径的两个端点在圆上，圆心是直径的中点。直径将圆分为面积相等的两部分，中间的线段就叫直径。.Question:知周长求直径的公式是什么',
  '知周长求直径的公式是直径是指通过一平面图形或立体中心到边上两点间的距离，通常用字母[UNK]d[UNK]表示'),
 ('N545561',
  'Paragraph:比如：精神病人、僵尸、女鬼等等。NPC的存在不仅能够渲染游戏中紧张的氛围，还能够在玩家进行不下去的时候给出线索，帮助玩家完成闯关，玩家通关之后，快速还

In [3]:
def load_prediction_BERT_short(path, final_qas_map, final_que_map, final_doc_map,shot_ans):
    prediction_result_100 = []
    prediction_result_60 = []
    prediction_not_null_60 = []
    prediction_not_null_100 = []
    final_text=[]
    final_answer = []
    final_qid = []
    final_text2=[]
    ground_truth = []
    final_answer2 = []
    final_qid2 = []
    with open(path, "r",encoding='utf-8') as f:
        all_data = f.readlines()
        for line in tqdm(all_data):
            try:
                mark = 0
                qas_id = line.split('	')[0]
                current_object_list = json.loads(line.split('	')[1])
                question = final_que_map[qas_id]
                for q in shot_ans:
                    if q in question:
                        mark = 1
                        break
                if mark == 1:
                    max_index = 0
                    max_prob = 0
                    for index, element in enumerate(current_object_list):
                        if element['probability'] > max_prob:
                            max_index = index
                        max_prob = max(max_prob, element['probability'])
                    prediction = current_object_list[max_index]['text']
                    actual_answer = final_qas_map[qas_id]
                    actual_answer_final = actual_answer
                    if actual_answer == prediction and actual_answer == '':
                        prediction_result_100.append(1)
                        prediction_result_60.append(1)
                    else:
                        overlap_rate = find_lcsubstr(actual_answer, prediction)
                        if overlap_rate == 1: # 考虑包含情况
                            prediction_result_100.append(1)
                        else:
                            prediction_result_100.append(0)
                        if overlap_rate >=0.6: #考虑包含情况
                            prediction_result_60.append(1)
                        else:
                            prediction_result_60.append(0)
        #             (prediction in answer and len(prediction)/len(answer)>=0.5)
                    if actual_answer_final!='':
                        overlap_rate = find_lcsubstr(actual_answer, prediction)
                        if overlap_rate == 1:
                            prediction_not_null_100.append(1)
                        else:
                            prediction_not_null_100.append(0)
                        if overlap_rate >= 0.6:
                            prediction_not_null_60.append(1)
                        else:
                            prediction_not_null_60.append(0)

                    if actual_answer_final == '' and prediction != '':
                        text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                        final_text.append(text)
                        final_answer.append(prediction)
                        final_qid.append(qas_id)

                    if actual_answer != '' and prediction != actual_answer:
                        text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                        final_text2.append(text)
                        ground_truth.append(actual_answer)
                        final_answer2.append(prediction)
                        final_qid2.append(qas_id)
            except KeyError:
                m = 1
        print('BERT short prediction, total length:{}'.format(len(prediction_result_100)))
        print('100% 相同情况下 准确率：', prediction_result_100.count(1)/len(prediction_result_100))
        print('60% 相同情况下 准确率：', prediction_result_60.count(1)/len(prediction_result_60))
        print('非空总数：{}'.format(len(prediction_not_null_100)))
        print('100% 相同情况下 答案非空 准确率：', prediction_not_null_100.count(1)/len(prediction_not_null_100))
        print('60% 相同情况下 答案非空 准确率：', prediction_not_null_60.count(1)/len(prediction_not_null_60))
        final_dic = list(zip(final_qid, final_text, final_answer))
        final_dic2 = list(zip(final_qid2, final_text2, final_answer2, ground_truth))
        return final_dic,final_dic2

In [35]:
suspicious_BERT_short, nonconsistent_BERT_short = load_prediction_BERT_short('nbest_predictions.json', final_qas_map,final_que_map, final_doc_map,shot_ans)

100%|█████████████████████████████████████████████████████████████████████████| 63369/63369 [00:02<00:00, 29108.22it/s]

BERT short prediction, total length:46278
100% 相同情况下 准确率： 0.8139288646873244
60% 相同情况下 准确率： 0.8154198539262717
非空总数：28752
100% 相同情况下 答案非空 准确率： 0.7895798553144129
60% 相同情况下 答案非空 准确率： 0.7919796883695047


In [73]:
def load_prediction_BERT_long(path, final_qas_map, final_que_map, final_doc_map,shot_ans):
    prediction_result_100 = []
    prediction_result_60 = []
    prediction_not_null_60 = []
    prediction_not_null_100 = []
    final_text=[]
    final_answer = []
    final_qid = []
    final_text2=[]
    ground_truth = []
    final_answer2 = []	
    final_qid2 = []
    with open(path, "r",encoding='utf-8') as f:
        all_data = f.readlines()
        for line in tqdm(all_data):
            try:
                mark = 0
                qas_id = line.split('	')[0]
                current_object_list = json.loads(line.split('	')[1])
                question = final_que_map[qas_id]
                for q in shot_ans:
                    if q in question:
                        mark = 1
                        break
                if mark == 0:
                    max_index = 0
                    max_prob = 0
                    for index, element in enumerate(current_object_list):
                        if element['probability'] > max_prob:
                            max_index = index
                        max_prob = max(max_prob, element['probability'])
                    prediction = current_object_list[max_index]['text']
                    actual_answer = final_qas_map[qas_id]
                    actual_answer_final = actual_answer
                    if actual_answer == prediction and actual_answer == '':
                        prediction_result_100.append(1)
                        prediction_result_60.append(1)
                    else:
                        overlap_rate = find_lcsubstr(actual_answer, prediction)
                        if overlap_rate == 1: # 考虑包含情况
                            prediction_result_100.append(1)
                        else:
                            prediction_result_100.append(0)
                        if overlap_rate >=0.6: #考虑包含情况
                            prediction_result_60.append(1)
                        else:
                            prediction_result_60.append(0)
        #             (prediction in answer and len(prediction)/len(answer)>=0.5)
                    if actual_answer_final!='':
                        overlap_rate = find_lcsubstr(actual_answer, prediction)
                        if overlap_rate == 1:
                            prediction_not_null_100.append(1)
                        else:
                            prediction_not_null_100.append(0)
                        if overlap_rate >= 0.6:
                            prediction_not_null_60.append(1)
                        else:
                            prediction_not_null_60.append(0)

                    if actual_answer_final == '' and prediction != '':
                        text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                        final_text.append(text)
                        final_answer.append(prediction)
                        final_qid.append(qas_id)

                    if actual_answer != '' and prediction != actual_answer:
                        text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                        final_text2.append(text)
                        ground_truth.append(actual_answer)
                        final_answer2.append(prediction)
                        final_qid2.append(qas_id)
            except KeyError:
                m = 1
        print('BERT long prediction, total length:{}'.format(len(prediction_result_100)))
        print('100% 相同情况下 准确率：', prediction_result_100.count(1)/len(prediction_result_100))
        print('60% 相同情况下 准确率：', prediction_result_60.count(1)/len(prediction_result_60))
        print('非空总数：{}'.format(len(prediction_not_null_100)))
        print('100% 相同情况下 答案非空 准确率：', prediction_not_null_100.count(1)/len(prediction_not_null_100))
        print('60% 相同情况下 答案非空 准确率：', prediction_not_null_60.count(1)/len(prediction_not_null_60))
        final_dic = list(zip(final_qid, final_text, final_answer))
        final_dic2 = list(zip(final_qid2, final_text2, final_answer2, ground_truth))
        return final_dic,final_dic2

In [37]:
suspicious_BERT_long, nonconsistent_BERT_long = load_prediction_BERT_long('nbest_predictions.json', final_qas_map,final_que_map, final_doc_map,shot_ans)

100%|█████████████████████████████████████████████████████████████████████████| 63369/63369 [00:01<00:00, 31972.28it/s]

BERT long prediction, total length:17091
100% 相同情况下 准确率： 0.7690012287168685
60% 相同情况下 准确率： 0.771926745070505
非空总数：10071
100% 相同情况下 答案非空 准确率： 0.7361731704895244
60% 相同情况下 答案非空 准确率： 0.7411379207625857


In [5]:
suspicious_ROBERTA_long, nonconsistent_ROBERTA_long = load_prediction_RoBERTA_long(evaluate_file, final_qas_map,final_que_map, final_doc_map,shot_ans)

NameError: name 'load_prediction_RoBERTA_long' is not defined

In [6]:
def load_prediction_RoBERTA(path, final_qas_map,final_que_map, final_doc_map):
    final_text=[]
    final_answer = []
    final_qid = []
    final_text2=[]
    ground_truth = []
    final_answer2 = []
    final_qid2 = []
    prediction_result_100 = []
    prediction_result_60 = []
    prediction_not_null_60 = []
    prediction_not_null_100 = []
    count = 0
    with open(path, "r",encoding='utf-8') as f:
        all_data = f.readlines()
        for line in tqdm(all_data):
            current_object = json.loads(line)
            qas_id = current_object['question_ID']
            actual_answer = ''.join(current_object['answer'].split())
            actual_answer_final = final_qas_map[qas_id]
            prediction = ''.join(current_object['prediction'].split())
            if actual_answer == prediction and actual_answer == '':
                prediction_result_100.append(1)
                prediction_result_60.append(1)
            else:
                overlap_rate = find_lcsubstr(actual_answer, prediction)
                if overlap_rate == 1: # 考虑包含情况
                    prediction_result_100.append(1)
                else:
                    prediction_result_100.append(0)
                if overlap_rate >=0.6: #考虑包含情况
                    prediction_result_60.append(1)
                else:
                    prediction_result_60.append(0)
#             (prediction in answer and len(prediction)/len(answer)>=0.5)
            if actual_answer_final!='':
                overlap_rate = find_lcsubstr(actual_answer, prediction)
                if overlap_rate == 1:
                    prediction_not_null_100.append(1)
                else:
                    prediction_not_null_100.append(0)
                if overlap_rate >= 0.6:
                    prediction_not_null_60.append(1)
                else:
                    prediction_not_null_60.append(0)
                    
            if actual_answer_final!='':
                    count +=1
            if actual_answer == '' and prediction != '':
                text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                final_text.append(text)
                final_answer.append(prediction)
                final_qid.append(qas_id)
                
            if actual_answer != '' and prediction != actual_answer:
                text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                final_text2.append(text)
                ground_truth.append(actual_answer_final)
                final_answer2.append(prediction)
                final_qid2.append(qas_id)
        print('RoBERTa all prediction, total length:{}'.format(len(prediction_result_100)))
        print('100% 相同情况下 准确率：', prediction_result_100.count(1)/len(prediction_result_100))
        print('60% 相同情况下 准确率：', prediction_result_60.count(1)/len(prediction_result_60))
        print('非空总数：{}'.format(count))
        print('100% 相同情况下 答案非空 准确率：', prediction_not_null_100.count(1)/len(prediction_not_null_100))
        print('60% 相同情况下 答案非空 准确率：', prediction_not_null_60.count(1)/len(prediction_not_null_60))
        final_dic = list(zip(final_qid, final_text, final_answer))
        final_dic2 = list(zip(final_qid2, final_text2, final_answer2, ground_truth))
        return final_dic,final_dic2

In [69]:
def load_prediction_BERT(path, final_qas_map, final_que_map, final_doc_map):
    prediction_result_100 = []
    prediction_result_60 = []
    prediction_not_null_60 = []
    prediction_not_null_100 = []
    final_text=[]
    final_answer = []
    final_qid = []
    final_text2=[]
    ground_truth = []
    final_answer2 = []
    final_qid2 = []

    with open(path, "r",encoding='utf-8') as f:
        all_data = f.readlines()
        for line in tqdm(all_data):
            try:
                qas_id = line.split('	')[0]
                current_object_list = json.loads(line.split('	')[1])
                max_index = 0
                max_prob = 0
                for index, element in enumerate(current_object_list):
                    if element['probability'] > max_prob:
                        max_index = index
                    max_prob = max(max_prob, element['probability'])

                prediction = current_object_list[max_index]['text']
                actual_answer = final_qas_map[qas_id]
                actual_answer_final = actual_answer

                if actual_answer == prediction and actual_answer == '':
                    prediction_result_100.append(1)
                    prediction_result_60.append(1)
                else:
                    overlap_rate = find_lcsubstr(actual_answer, prediction)
                    if overlap_rate == 1: # 考虑包含情况
                        prediction_result_100.append(1)
                    else:
                        prediction_result_100.append(0)
                    if overlap_rate >=0.6: #考虑包含情况
                        prediction_result_60.append(1)
                    else:
                        prediction_result_60.append(0)
    #             (prediction in answer and len(prediction)/len(answer)>=0.5)
                if actual_answer_final!='':
                    overlap_rate = find_lcsubstr(actual_answer, prediction)
                    if overlap_rate == 1:
                        prediction_not_null_100.append(1)
                    else:
                        prediction_not_null_100.append(0)
                    if overlap_rate >= 0.6:
                        prediction_not_null_60.append(1)
                    else:
                        prediction_not_null_60.append(0)

                if actual_answer_final == '' and prediction != '':

                    text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])

                    final_text.append(text)
                    final_answer.append(prediction)
                    final_qid.append(qas_id)

                if actual_answer != '' and prediction != actual_answer:
                    text =  "Paragraph:{}.Question:{}".format(final_doc_map[qas_id],final_que_map[qas_id])
                    final_text2.append(text)
                    ground_truth.append(actual_answer)
                    final_answer2.append(prediction)
                    final_qid2.append(qas_id)
            except KeyError:
                m = 1
        print('BERT all prediction, total length:{}'.format(len(prediction_result_100)))
        print('100% 相同情况下 准确率：', prediction_result_100.count(1)/len(prediction_result_100))
        print('60% 相同情况下 准确率：', prediction_result_60.count(1)/len(prediction_result_60))
        print('非空总数：{}'.format(len(prediction_not_null_100)))
        print('100% 相同情况下 答案非空 准确率：', prediction_not_null_100.count(1)/len(prediction_not_null_100))
        print('60% 相同情况下 答案非空 准确率：', prediction_not_null_60.count(1)/len(prediction_not_null_60))
        final_dic = list(zip(final_qid, final_text, final_answer))
        final_dic2 = list(zip(final_qid2, final_text2, final_answer2, ground_truth))
        return final_dic,final_dic2

In [41]:
suspicious_BERT, nonconsistent_BERT = load_prediction_BERT('nbest_predictions.json', final_qas_map,final_que_map, final_doc_map)

100%|█████████████████████████████████████████████████████████████████████████| 63369/63369 [00:02<00:00, 25582.98it/s]

BERT all prediction, total length:63369
100% 相同情况下 准确率： 0.8018116113557102
60% 相同情况下 准确率： 0.8036895011756537
非空总数：38823
100% 相同情况下 答案非空 准确率： 0.7757257295932823
60% 相同情况下 答案非空 准确率： 0.7787909229065245


In [7]:
suspicious_ROBERTA, nonconsistent_ROBERTA = load_prediction_RoBERTA(evaluate_file, final_qas_map,final_que_map, final_doc_map)

100%|████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 1670.78it/s]

RoBERTa all prediction, total length:1500
100% 相同情况下 准确率： 0.9533333333333334
60% 相同情况下 准确率： 0.9586666666666667
非空总数：1297
100% 相同情况下 答案非空 准确率： 0.9491133384734002
60% 相同情况下 答案非空 准确率： 0.9552814186584425


In [70]:
import json
from tqdm import tqdm
import pandas as pd

def find_lcsubstr(str_a, str_b):
#  longest common subsequence of str_a and str_b, with O(n) space complexity
    if len(str_a) == 0 or len(str_b) == 0:
        return 0
    dp = [0 for _ in range(len(str_b) + 1)]
    for i in range(1, len(str_a) + 1):
        left_up = 0
        dp[0] = 0
        for j in range(1, len(str_b) + 1):
            left = dp[j-1]
            up = dp[j]
            if str_a[i-1] == str_b[j-1]:
                dp[j] = left_up + 1
            else:
                dp[j] = max([left, up])
            left_up = up
    return dp[len(str_b)]/min(len(str_b),len(str_a))

def data_loading(path):
    all_keys = ['question_id','question','qas_id','doc_tokens','answer_text','start_position','end_position','is_impossible']
    all_values = [[] for i in range(len(all_keys))]
    final_dic = dict(zip(all_keys,all_values))
    with open(path,encoding='utf-8') as json_file:
        for line in tqdm(json_file):
            data = json.loads(line)
            current_keys = data.keys()
            for key in current_keys:
                final_dic[key].append(str(data[key]))
    final_pd = pd.DataFrame(final_dic)
    final_pd.start_position = final_pd.start_position.replace({-1:0})
    final_pd.end_position = final_pd.end_position.replace({-1:0})
    return final_pd

test_df_location = 'test_new/bert_tmp_test_3.0_data.json'
test_df = data_loading(test_df_location)
all_qas_id = list(test_df['qas_id'])
all_question = list(test_df['question'])
all_doc_tokens = list(test_df['doc_tokens'])
answer_all = list(test_df['answer_text'])
final_qas_map = dict(zip(all_qas_id,answer_all))
final_que_map = dict(zip(all_qas_id,all_question))
final_doc_map = dict(zip(all_qas_id,all_doc_tokens))
shot_ans = ['是什么','在哪里','哪位','哪国','国家','国籍','哪个朝代','本名','原名','哪个','哪一个','哪','最','几次','是谁','学名','第一个','第一','最大','最小','最长','最深','最高','最多','第一','作者','叫什么名字','名字','叫什么']

6243it [00:00, 84389.47it/s]


In [74]:
suspicious_BERT, nonconsistent_BERT = load_prediction_BERT('test_new/nbest_predictions_ (3).json', final_qas_map,final_que_map, final_doc_map)
suspicious_BERT_long, nonconsistent_BERT_long = load_prediction_BERT_long('test_new/nbest_predictions_ (3).json', final_qas_map,final_que_map, final_doc_map,shot_ans)
suspicious_BERT_short, nonconsistent_BERT_short = load_prediction_BERT_short('test_new/nbest_predictions_ (3).json', final_qas_map,final_que_map, final_doc_map,shot_ans)

 16%|███████████▊                                                               | 1501/9519 [00:00<00:00, 14861.13it/s]

BERT all prediction, total length:9321
100% 相同情况下 准确率： 0.6615170046132389
60% 相同情况下 准确率： 0.6771805600257483
非空总数：9313
100% 相同情况下 答案非空 准确率： 0.6618705035971223
60% 相同情况下 答案非空 准确率： 0.677547514227424


 18%|█████████████▎                                                             | 1683/9519 [00:00<00:00, 16824.16it/s]

BERT long prediction, total length:6256
100% 相同情况下 准确率： 0.665920716112532
60% 相同情况下 准确率： 0.6801470588235294
非空总数：6252
100% 相同情况下 答案非空 准确率： 0.6660268714011516
60% 相同情况下 答案非空 准确率： 0.6802623160588611


100%|███████████████████████████████████████████████████████████████████████████| 9519/9519 [00:00<00:00, 16641.65it/s]

BERT short prediction, total length:3065
100% 相同情况下 准确率： 0.6525285481239804
60% 相同情况下 准确率： 0.6711256117455139
非空总数：3061
100% 相同情况下 答案非空 准确率： 0.6533812479581836
60% 相同情况下 答案非空 准确率： 0.6720026135249918


## Save your target files

In [6]:
def prediction_saving(path, suspicious_list):
    with open(path, "w",encoding='utf-8') as f:
        for index in range(len(suspicious_list)):
            if len(suspicious_list[index]) == 3:
                aDict = {"QID": suspicious_list[index][0], "Text":suspicious_list[index][1], "prediction": suspicious_list[index][2]}
                jsonString = json.dumps(aDict,ensure_ascii=False)
                f.write(jsonString)
                f.write('\n')
            elif len(suspicious_list[index]) == 4:
                aDict = {"QID": suspicious_list[index][0], "Text":suspicious_list[index][1], "answer":suspicious_list[index][3],
                         "prediction": suspicious_list[index][2]}
                jsonString = json.dumps(aDict,ensure_ascii=False)
                f.write(jsonString)
                f.write('\n')

In [7]:
prediction_saving('BERT_error.json', suspicious_BERT)
prediction_saving('RoBERTa_error.json', suspicious_ROBERTA)
prediction_saving('BERT_inconsist_error.json', nonconsistent_BERT)
prediction_saving('RoBERTa_inconsist_error.json', nonconsistent_ROBERTA)

In [8]:
def convert_zip_to_dic(suspicious_list):
    final_dic = {}
    for index in range(len(suspicious_list)):
        temp_dic = {"Text":suspicious_list[index][1], "prediction": suspicious_list[index][2]}
        final_dic[suspicious_list[index][0]] = temp_dic
    return final_dic

In [9]:
BERT_dic = convert_zip_to_dic(suspicious_BERT)
RoBERTa_dic = convert_zip_to_dic(suspicious_ROBERTA)

In [10]:
def check_same_ID(BERT, RoBERTA):
    final_group = []
    for key in BERT.keys():
        try:
            roberta_content = RoBERTA[key]["prediction"]
            bert_content = BERT[key]["prediction"]
            text = BERT[key]["Text"]
            temp_dic = {'QID':key, 'Text':text, 'BERT_prediction':bert_content,'RoBERTa_prediction':roberta_content}
            final_group.append(temp_dic)
        except KeyError:
            m = 0
    return final_group

In [11]:
final_consistent_check = check_same_ID(BERT_dic, RoBERTa_dic)

In [12]:
with open('B&R_consistency.json', "w",encoding='utf-8') as f:
    for dic in final_consistent_check:
        jsonString = json.dumps(dic,ensure_ascii=False)
        f.write(jsonString)
        f.write('\n')

In [13]:
final_consistent_check

[{'QID': 'Q_IR_VAL_000006#03',
  'Text': 'Paragraph:1815年，在滑铁卢战役失败后，拿破仑被流放于南大西洋的圣赫勒拿岛，并于6年后死于该岛，终年52岁.Question:拿破仑死于哪一年',
  'BERT_prediction': '1815',
  'RoBERTa_prediction': '1815'},
 {'QID': 'Q_IR_VAL_000006#07',
  'Text': 'Paragraph:1815年，在滑铁卢战役失败后，拿破仑被流放于南大西洋的圣赫勒拿岛，并于六年之后死于该岛，终年52岁.Question:拿破仑死于哪一年',
  'BERT_prediction': '1815',
  'RoBERTa_prediction': '1815'},
 {'QID': 'Q_IR_VAL_000008#03',
  'Text': 'Paragraph:宜饭时服用的药物常见的有鱼肝油等脂溶性药物，因为脂溶性药物可溶解于食物中的脂肪内，更容易吸收；另外，助消化药与食物同时服用，其助消化作用更佳.Question:助消化的药物应在什么时候服用？',
  'BERT_prediction': '饭时',
  'RoBERTa_prediction': '宜饭时服用的药物常见的有鱼肝油等脂溶性药物，因为脂溶性药物可溶解于食物中的脂肪内，更容易吸收；另外，助消化药与食物同时服用，其助消化作用更佳'},
 {'QID': 'Q_IR_VAL_000008#16',
  'Text': 'Paragraph:助消化药，如酵母、乳酶生、胰酶等，饭间服用可及时发挥药效.Question:助消化的药物应在什么时候服用？',
  'BERT_prediction': '饭间',
  'RoBERTa_prediction': '饭间'},
 {'QID': 'Q_IR_VAL_000008#17',
  'Text': 'Paragraph:饭前服用”一般是指饭前半小时服用，健胃药、助消化药大都在饭前服用.Question:助消化的药物应在什么时候服用？',
  'BERT_prediction': '饭前',
  'RoBERTa_prediction': '饭前'},
 {'QID': 'Q_IR_VAL_